In [1]:
import ipywidgets as widgets
from SPARQLWrapper import SPARQLWrapper, JSON
import shutil
import os

class HealthDM(object):
    profile = None
    profilePresent = None
    profileAbsent = None
    
    sparqlQuery = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
PREFIX mesh2015: <http://id.nlm.nih.gov/mesh/2015/>
PREFIX mesh2016: <http://id.nlm.nih.gov/mesh/2016/>
PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>

SELECT ?d
FROM <http://id.nlm.nih.gov/mesh>
WHERE {{
    ?d a meshv:Term .
    {{?d meshv:prefLabel ?l
     FILTER (lcase(str(?l)) = lcase('{0}'))}}
    UNION
    {{?d meshv:altLabel ?l
     FILTER (lcase(str(?l)) = lcase('{0}'))}}
}}
ORDER BY ?d
"""
    
    # @classmethod
    # def setProfile(cls, profile):
    #     cls.profile = profile
    
    @classmethod
    def createProfile(cls):
        cls.profile = widgets.Accordion(children=[])
        return cls.profile
        
    @classmethod
    def clearTerms(cls):
        cls.profilePresent = widgets.Accordion(children=[])
        cls.profileAbsent = widgets.Accordion(children=[])
        cls.profile.children = tuple([cls.profilePresent, cls.profileAbsent])
        cls.profile.set_title(0, "Sign Present")
        cls.profile.set_title(1, "Sign Absent")

    @classmethod
    def addTerm(cls, heading, description, code, present, detail1, rate1, detail2, rate2):
        wprofile = [widgets.Text(description = "MeSH Term:", value = heading),
                    widgets.Text(description = "MeSH Code:", value = code)]
        if detail1 != "#":
            fDetail = cls.formatDetail(detail1, rate1)
            wprofile.append(widgets.Text(description = fDetail[0], value = fDetail[1]))
        if detail2 != "#":
            fDetail = cls.formatDetail(detail2, rate2)
            wprofile.append(widgets.Text(description = fDetail[0], value = fDetail[1]))
        wcontent = widgets.VBox(wprofile)
        profileList = cls.profilePresent if present else cls.profileAbsent
        profileList.children = tuple(list(profileList.children) + [wcontent])
        profileList.set_title(len(profileList.children)-1, description)
        
    @classmethod
    def formatDetail(cls, detail, rate):
        detailDesc = "Detail:"
        detailValue = detail
        if detail in ["occasionally", "often", "frequently", "sometimes",
                      "rarely", "daily", "monthly", "yearly", "each day",
                      "each month", "each year", "once a day", "once a month",
                      "once a year"]:
            detailDesc = "Frequency:"
        else:
            try:
                value = int(detail)
                detailDesc = "Intensity:"
                if rate != "#":
                    detailDesc = "Frequency:"
                    detailValue = value + " / " + rate
            except ValueError:
                pass
        return [detailDesc, detailValue]

    @classmethod
    def findMeshCode(cls, heading, detail1, rate1, description, detail2, rate2):
        code = "none"
        present = True
        meshHeading = heading
        
        sparql = SPARQLWrapper("http://id.nlm.nih.gov/mesh/sparql")
        sparql.setReturnFormat(JSON)

        # looking for the heading
        sparql.setQuery(cls.sparqlQuery.format(meshHeading))
        results = sparql.query().convert()
        if len(results["results"]["bindings"]) > 0:
            code = results["results"]["bindings"][0]["d"]["value"]

        if code == "none" and (heading.lower().startswith("no") or
                               heading.lower().startswith("not")):
            # looking for the heading
            present = False
            meshHeading = heading[3:] if heading.lower().startswith("no") else heading[4:]
            sparql.setQuery(cls.sparqlQuery.format(meshHeading))
            results = sparql.query().convert()
            if len(results["results"]["bindings"]) > 0:
                code = results["results"]["bindings"][0]["d"]["value"]
            
        cls.addTerm(meshHeading, description, code, present, detail1, rate1, detail2, rate2)
    
        return "#mesh_heading#" + heading + "#tree_number#" + code
    
    @classmethod
    def interfaceMain(cls, title, description, image, firstKnot):
        shutil.rmtree("html")
        os.mkdir("html")
        dirs = ["css", "js"]
        for d in dirs:
            shutil.copytree("template/{}".format(d), "html/{}".format(d))
        shutil.copytree("images", "html/images")
        files = ["index", "start", "signin", "register", "report"]
        for f in files:
            shutil.copyfile("template/{}.html".format(f), "html/{}.html".format(f))
        indexTemplate = open("template/casesindex.html", "r", encoding="utf-8")
        indexResult = open("html/casesindex.html", "w", encoding="utf-8")
        indexResult.write(
            indexTemplate.read().format(title=title, description=description, image=image, firstKnot=firstKnot))
        indexTemplate.close()
        indexResult.close()

    @classmethod
    def interfaceKnot(cls, template, htmlName, title, description, image):
        knotTemplate = open("template/{}.html".format(template), "r", encoding="utf-8")
        knotResult = open("html/" + htmlName + ".html", "w", encoding="utf-8")
        knotResult.write(knotTemplate.read().format(title=title, description=description, image=image))
        knotResult.close()

HealthDM.createProfile()

Accordion()

== Case 001 (dialog) ==
![Nurse](images/nurse.png)
:Nurse: Doctor, please you have to evaluate a man (51 years-old) who entered the emergency department reporting chest pain. His vital signs are ABP: 144x92mmHG; HR: 78bpm; RR: 21rpm; Temp: 37oC; O2Sat: 98%.

:Patient: Doctor, I am feeling chest pain since yesterday. The pain is continuous and is located just in the middle of my chest, worsening when I breath and when I lay down on my bed. I have arterial hypertension and I {smoke} 20 cigarettes every day. My father had a “heart attack” at my age and I am very worried about it.

Physical examination: cardiac and pulmonary auscultation are normal; chest pain does not worse with palpation of the thorax; there is no jugular stasis nor lower limb edema.

Jacinto: What do you want to do?
++ Generate hypothesis -> Generate hypothesis 1
++ More information -> More information 1
++ Call the supervisor -> Call the supervisor 1

== Generate hypothesis 1 ==
What is your main diagnostic hypothesis?
{?1 hypothesis:mesh}
++ Submit hypothesis -> Check hypothesis 1

== More information 1 (notice) ==
The patient never felt chest pain before. He exercises regularly and has lost weight in the last three months. 

He takes amlodipine and losartan regularly.

Two weeks ago, he had an auto-limited gastroenteritis episode.
He denies recent travels and surgery.

== Call the supervisor 1 (notice) ==
Chest pain is one of the commonest complains referred by patients at the emergency department. The first step is to exclude or to confirm if the patient has one of the five diseases which can have as the main complaint chest pain, and are related to sudden death: myocardial infarction, aortic dissection, pulmonary embolism, hypertensive pneumothorax, and Boerhaave Syndrome.

To investigate those diseases, we need to analyze the clinical history and physical examination data, as some of them can increase or reduce the probability of each disease as we can see in the following tables:
++ Clinical History Myocardial infarction
++ Physical examination myocardial infarction
++ Clinical History Aortic Dissection
++ Physical Examination Aortic Dissection
++ Pulmonary embolism Wells criteria

Hypertensive pneumothorax is more common in longelineous young adults (primary pneumothorax) or in patients with chronic pulmonary diseases or chest trauma (secondary pneumothorax). On Physical examination, we expect asymmetry in lung auscultation and the trachea may be dislocated to the contralateral side of the pneumothorax.

Boerhhave Syndrome is more common in patients who presented vomiting before chest pain, were submitted to endoscopic procedures or had chest trauma.
As our patient has hypertension, we could be worried about aortic dissection, but there is no other data that increase it´s likelihood ratio. Also, there is no data that increases myocardial infarction likelihood ratio. Also, our patient has no feature of Wells criteria for now, but it´s is important to ask him about immobilization, trips or recent surgery.

== Clinical History Myocardial infarction (notice) ==
![History Infarction](images/history-infarction.png)

== Check hypothesis 1 ==
Among all patient features, signs and symptoms highlighted on text, select all that increase the probability of your diagnostic hypothesis:

Nurse: Doctor, please you have to evaluate a man (51 years-old) who entered the emergency department reporting chest pain. His vital signs are ABP: 144x92mmHG; HR: 78bpm; {RR: 21rpm}(p); {Temp: 37oC}(i); O2Sat: 98%.

Patient: Doctor, I am feeling {chest pain} since yesterday. The pain is continuous and is located just in the middle of my chest, worsening when I breath and when I lay down on my bed. I have arterial hypertension and I smoke 20 cigarettes every day. My father had a “heart attack” at my age and I am very worried about it. 
You perform physical examination: cardiac and pulmonary auscultation are normal; chest pain does not worse with palpation of the thorax; there is no jugular stasis nor lower limb edema.
++ Submit -> Order EKG 

== Order EKG ==
Patient deny any recent long trip, immobilization or surgery.
You re-examine the patient, obtaining the blood pressure in the four limbs, and you don´t find any significant differences. 
An EKG is then ordered:
![EKG](images/ekg2.png)
Game: What do you want to do?
++ Generate hypothesis -> Generate hypothesis 2
++ Get more information -> More information 2
++ Call the supervisor -> Call the supervisor 2

== Generate hypothesis 2 ==
What is your main diagnostic hypothesis?
{?1 hypothesis:mesh}
++ Submit hypothesis -> Check hypothesis 2

== More information 2 (notice) ==
![EKG-A](images/ampliacao-eletro.gif)

== Call the supervisor 2 (notice) ==
![EKG-A](images/ampliacao-eletro.gif)
We did not find other features that increase the likelihood of the 5 causes of chest pain associated with sudden death. Moreover, our patient has a pleuritic chest pain which worse when the patient lay down. This feature increases the probability of pericardial diseases.

In EKG we found ST-segment elevation in almost all leads. Also we found PR interval “infra” in DII lead.

These EKG findings, associated to clinical history and physical examination, are strongly associated to a specific diagnostic hypothesis.

== Check hypothesis 2 ==
![EKG-A](images/ampliacao-eletro.gif)
++ Submit -> Final report

== Final report ==
# Feedback
Score: ...
Evaluation: ...
Two columns

In [2]:
%%HTML
<iframe width="100%" height="800px" src="html/index.html"><iframe>